In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, Sequential, datasets, optimizers
import os


In [5]:
# config = tf.ConfigProto()
# tf.gpu._options.per_process_gpu_memory_fraction = 0.5
# config.gpu_options.allow_growth = True

AttributeError: module 'tensorflow' has no attribute 'gpu'

In [1]:


# convolution layers
conv_layers = [
    # 5 units of conv + max poling
    # unit 1
    layers.Conv2D(64, kernel_size=[3, 3], padding='same', activation=tf.nn.relu),
#     layers.Conv2D(64, kernel_size=[3, 3], padding='same', activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),
    # unit 2
#     layers.Conv2D(128, kernel_size=[3, 3], padding='same', activation=tf.nn.relu),
#     layers.Conv2D(128, kernel_size=[3, 3], padding='same', activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),
    # unit 3
#     layers.Conv2D(256, kernel_size=[3, 3], padding='same', activation=tf.nn.relu),
#     layers.Conv2D(256, kernel_size=[3, 3], padding='same', activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),
    # unit 4
#     layers.Conv2D(512, kernel_size=[3, 3], padding='same', activation=tf.nn.relu),
#     layers.Conv2D(512, kernel_size=[3, 3], padding='same', activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),
    # unit 5
#     layers.Conv2D(512, kernel_size=[3, 3], padding='same', activation=tf.nn.relu),
#     layers.Conv2D(512, kernel_size=[3, 3], padding='same', activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same')
    
]
conv_net = Sequential(conv_layers)

# full connection layers
fc_net = Sequential([
    layers.Dense(512, activation=tf.nn.relu),    # [b, 512] => [b, 512]
    layers.Dense(256, activation=tf.nn.relu),    # [b, 512] => [b, 256]
    layers.Dense(100, activation=None)           # [b, 256] => [b, 100]
])

# 定义预处理函数
def preprocess(x, y):
    # [0~1]
    x = tf.cast(x, dtype=tf.float32) / 255.
    y = tf.cast(y, dtype=tf.int32)
    return x, y
# 创建网络层
conv_net.build(input_shape=[None, 32, 32, 3])
fc_net.build(input_shape=[None, 512])
# 定义优化器
optimizer = optimizers.Adam(lr=1e-4)
# 定义网络成参数列表
variables = conv_net.trainable_variables + fc_net.trainable_variables


# 加载数据集和数据集预处理
(x, y), (x_test, y_test) = datasets.cifar100.load_data()
y = tf.squeeze(y, axis=1)
y_test = tf.squeeze(y_test, axis=1)

train_db = tf.data.Dataset.from_tensor_slices((x, y))
train_db = train_db.shuffle(10000).map(preprocess).batch(128)
# test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test))
# test_db = test_db.map(preprocess).batch(128)

# 查看数据集切片
sample = next(iter(train_db))
print('sample_train: ', sample[0].shape, sample[1].shape, tf.reduce_min(sample[0]),
     tf.reduce_max(sample[1]))


for epoch in range(50):
    for step, (x, y) in enumerate(train_db):
        with tf.GradientTape() as tape:
            out = conv_net(x)    # [b, 32, 32, 32, 3] => [b, 1, 1, 512]
            out = tf.reshape(out, [-1, 512])    # [b, 1, 1, 512] => [b, 512]
            logits = fc_net(out)    # [b, 512] => [b, 100]
            y_onehot = tf.one_hot(y, depth=10)
            # compute loss
            loss = tf.losses.categorical_crossentropy(y_onehot, logits, from_logits=True)
        
        # compute grades    
        grads = tape.gradient(loss, variables)
        # update variables 
        optimizer.apply_gradients(zip(grads, variables))
        
        # show loss
        if step % 100 == 0:
            print(epoch, step, 'loss: ', float(loss))
            
#     # test    
#     total_num = 0
#     total_correct = 0
#     for x, y in test_db:
#         out = conv_net(x)
#         out = tf.reshape(out, [-1, 512])
#         logits = fc_net(out)
#         prob = tf.nn.softmax(logits, axis=1)
#         preb = tf.argmax(prob, axis=1)    # int64
#         preb = tf.cast(preb, dtype=tf.int32)
        
#         correct = tf.cast(tf.equal(perb, y), dtype=tf.int32)
#         correct = tf.reduce_sum(correct)
        
#         total_num += x.shape[0]
#         total_correct += int(correct)
        
#     acc = total_correct / total_num
#     print(epoch, 'acc: ', acc)
    


sample_train:  (128, 32, 32, 3) (128,) tf.Tensor(0.0, shape=(), dtype=float32) tf.Tensor(96, shape=(), dtype=int32)


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above. [Op:Conv2D]